In [1]:
import pandas as pd
import math
import numpy as np

%matplotlib inline

from pathlib import Path
from omegaconf import OmegaConf
import scipy.stats as st

In [2]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "valid_log.csv").is_file():
        cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
        cfg = OmegaConf.to_container(cfg)
        num_epochs = cfg['optim']['epochs']

        valid_log = pd.read_csv(run / "valid_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-3) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("valid_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    len(list(Path('.').glob('*')))
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def check_image_size(run, image_size=480):
    cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
    cfg = OmegaConf.to_container(cfg)
    conf_image_size = cfg['data']['image_size']
    if int(conf_image_size) != image_size:
        print("Different image size in conf. Image size in conf: {}, Image size: {}".format(conf_image_size, image_size))

def collect_one(model_name, run, csv_file, image_size=480):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    check_image_size(run, image_size=image_size)
    
    cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
    cfg = OmegaConf.to_container(cfg)

    run_number = cfg['data']['train']['cross_val_bucket_validation_index']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / 'test_predictions' / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = run_number
    data['inv_temp'] = inv_temp
    data['regime'] = regime
    
    return data

def collect_all(model_name, root, csv_file, regimes=['0.01', '0.02', '0.05', '0.1', '0.2', '0.25', '1.0'], image_size=480, ignore_outliers=True):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            if regime.name.rsplit("-", 1)[1] in regimes:
                for run in list(regime.glob("run-*")):
                    if ignore_outliers and 'outlier' in run.as_posix():
                        continue
                    else:
                        metrics.append(collect_one(model_name, run, csv_file, image_size=image_size))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

def collect_all_temperature(model_name, root, csv_file, regimes=['0.01', '0.02', '0.05', '0.1', '0.2', '0.25', '1.0'], image_size=480, ignore_outliers=True):
    root = Path(root)

    metrics = []
    for regime in list(root.glob("regime-*")):
        if regime.name.rsplit("-", 1)[1] in regimes:
            for run in list(regime.glob("run-*")):
                if ignore_outliers and 'outlier' in run.as_posix():
                    continue
                else:
                    metrics.append(collect_one(model_name, run, csv_file, image_size=image_size))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [3]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], model_group[1], model_group[2], model_group[3]
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90, return_ranges=False):

    def compute_ci(values, return_ranges=False):
        ci = st.t.interval(confidence_level, len(values)-1, loc=np.mean(values), scale=np.std(values)/(len(values)**0.5))

        if return_ranges:
            return ci
        else:
            return (ci[1]-ci[0]) / 2

    #mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)]) 

    return mean_metrics

<h1>GlaS Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [13]:
# Unsupervised learning models are evaluate over the whole dataset while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['1.0']
IMAGE_SIZE = 480

runs = {
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t')),
}

In [14]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv', regimes=REGIMES, image_size=IMAGE_SIZE, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [15]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,1.0,1.0,52.31,36.16,41.72,8.55
1,H-UNet-BASE-SWTA,0,5.0,1.0,49.74,33.87,42.40,8.16
2,H-UNet-BASE-SWTA,0,10.0,1.0,45.79,30.24,45.59,8.88
3,H-UNet-BASE-SWTA,0,20.0,1.0,49.81,33.74,38.32,7.20
4,H-UNet-BASE-SWTA,0,50.0,1.0,49.98,34.01,42.11,7.88
5,H-UNet-BASE-SWTA,0,100.0,1.0,49.48,33.46,40.05,7.27
6,H-UNet-BASE-SWTA-T,0,1.0,1.0,45.60,30.22,44.54,9.98
7,H-UNet-BASE-SWTA-T,0,5.0,1.0,48.07,32.26,54.12,10.52
8,H-UNet-BASE-SWTA-T,0,10.0,1.0,49.61,33.60,42.02,8.29
9,H-UNet-BASE-SWTA-T,0,20.0,1.0,50.11,34.03,42.04,7.84


/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a

Dice         Jaccard          \
                                     Mean CI 0.9%    Mean CI 0.9%   
Model              Inv Temp Regime                                  
H-UNet-BASE-SWTA   1.0      1.0     52.31     NaN   36.16     NaN   
                   5.0      1.0     49.74     NaN   33.87     NaN   
                   10.0     1.0     45.79     NaN   30.24     NaN   
                   20.0     1.0     49.81     NaN   33.74     NaN   
                   50.0     1.0     49.98     NaN   34.01     NaN   
                   100.0    1.0     49.48     NaN   33.46     NaN   
H-UNet-BASE-SWTA-T 1.0      1.0     45.60     NaN   30.22     NaN   
                   5.0      1.0     48.07     NaN   32.26     NaN   
                   10.0     1.0     49.61     NaN   33.60     NaN   
                   20.0     1.0     50.11     NaN   34.03     NaN   
                   50.0     1.0     47.67     NaN   31.89     NaN   
                   100.0    1.0     49.24     NaN   33.43     NaN   

                                   Hausdorff Distance          \
                                                 Mean CI 0.9%   
Model              Inv Temp Regime                              
H-UNet-BASE-SWTA   1.0      1.0                 41.72     NaN   
                   5.0      1.0                 42.40     NaN   
                   10.0     1.0                 45.59     NaN   
                   20.0     1.0                 38.32     NaN   
                   50.0     1.0                 42.11     NaN   
                   100.0    1.0                 40.05     NaN   
H-UNet-BASE-SWTA-T 1.0      1.0                 44.54     NaN   
                   5.0      1.0                 54.12     NaN   
                   10.0     1.0                 42.02     NaN   
                   20.0     1.0                 42.04     NaN   
                   50.0     1.0                 43.38     NaN   
                   100.0    1.0                 41.65     NaN   

                                   Average Surface Distance          
                                                       Mean CI 0.9%  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   1.0      1.0                        8.55     NaN  
                   5.0      1.0                        8.16     NaN  
                   10.0     1.0                        8.88     NaN  
                   20.0     1.0                        7.20     NaN  
                   50.0     1.0                        7.88     NaN  
                   100.0    1.0                        7.27     NaN  
H-UNet-BASE-SWTA-T 1.0      1.0                        9.98     NaN  
                   5.0      1.0                       10.52     NaN  
                   10.0     1.0                        8.29     NaN  
                   20.0     1.0                        7.84     NaN  
                   50.0     1.0                        8.20     NaN  
                   100.0    1.0                        8.29     NaN

In [34]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['0.01', '0.02', '0.05', '0.1', '0.2']
IMAGE_SIZE = 480

runs = {
    'H-UNet-BASE-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_ft')),
    'H-UNet-BASE-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t_ft')),
    'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_ft')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_t_ft')),
}

In [37]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [38]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA-FT,0,1.0,0.01,64.24,49.26,57.84,9.26
1,H-UNet-BASE-SWTA-FT,0,1.0,0.02,67.96,52.81,51.52,7.29
2,H-UNet-BASE-SWTA-FT,0,1.0,0.05,72.36,57.89,42.10,5.75
3,H-UNet-BASE-SWTA-FT,0,1.0,0.10,72.69,57.82,28.34,3.90
4,H-UNet-BASE-SWTA-FT,0,1.0,0.20,78.29,65.23,25.62,3.60
...,...,...,...,...,...,...,...,...
1095,H-UNet-Pseudolabeling-SWTA-T-FT,9,100.0,0.01,66.95,51.33,40.67,6.38
1096,H-UNet-Pseudolabeling-SWTA-T-FT,9,100.0,0.02,72.61,59.59,41.55,6.94
1097,H-UNet-Pseudolabeling-SWTA-T-FT,9,100.0,0.05,65.87,50.95,61.45,9.47
1098,H-UNet-Pseudolabeling-SWTA-T-FT,9,100.0,0.10,78.39,65.91,40.10,5.96


/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])


Dice           Jaccard  \
                                                   Mean   CI 0.9%    Mean   
Model                           Inv Temp Regime                             
H-UNet-BASE-SWTA-FT             1.0      0.01    65.742  0.977153  50.876   
                                         0.02    64.977  1.902475  49.779   
                                         0.05    70.160  1.702422  55.482   
                                         0.10    74.023  0.951953  60.101   
                                         0.20    77.890  1.098575  64.949   
...                                                 ...       ...     ...   
H-UNet-Pseudolabeling-SWTA-T-FT 100.0    0.01    66.874  1.864946  52.035   
                                         0.02    70.349  1.294908  56.447   
                                         0.05    72.386  2.303595  58.900   
                                         0.10    78.894  1.685691  66.768   
                                         0.20    84.492  1.171696  74.277   

                                                          Hausdorff Distance  \
                                                  CI 0.9%               Mean   
Model                           Inv Temp Regime                                
H-UNet-BASE-SWTA-FT             1.0      0.01    1.161506             56.590   
                                         0.02    2.011092             52.690   
                                         0.05    1.925587             45.307   
                                         0.10    1.271095             33.106   
                                         0.20    1.419287             28.680   
...                                                   ...                ...   
H-UNet-Pseudolabeling-SWTA-T-FT 100.0    0.01    2.057103             54.541   
                                         0.02    1.405975             50.192   
                                         0.05    2.854746             47.805   
                                         0.10    2.292143             35.750   
                                         0.20    1.629634             25.367   

                                                           \
                                                  CI 0.9%   
Model                           Inv Temp Regime             
H-UNet-BASE-SWTA-FT             1.0      0.01    2.489949   
                                         0.02    5.107129   
                                         0.05    4.653122   
                                         0.10    2.614397   
                                         0.20    1.919219   
...                                                   ...   
H-UNet-Pseudolabeling-SWTA-T-FT 100.0    0.01    4.040427   
                                         0.02    2.363189   
                                         0.05    5.100017   
                                         0.10    3.831478   
                                         0.20    2.354053   

                                                Average Surface Distance  \
                                                                    Mean   
Model                           Inv Temp Regime                            
H-UNet-BASE-SWTA-FT             1.0      0.01                      9.148   
                                         0.02                      8.353   
                                         0.05                      6.600   
                                         0.10                      4.906   
                                         0.20                      4.102   
...                                                                  ...   
H-UNet-Pseudolabeling-SWTA-T-FT 100.0    0.01                      8.364   
                                         0.02                      7.920   
                                         0.05                      7.322   
                                         0.10                      5.310   
        

<h2>Evaluation - Hebbian Unsupervised Pretraining</h2>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [39]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

IMAGE_SIZE = 480
INV_TEMP_GlaS=1          # to be set accordingly, used by SWTA

runs = {
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_GlaS)).glob('run-*')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_GlaS)).glob('run-*')),
}

In [40]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv', image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [41]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

/tmp/ipykernel_1451037/3550580355.py:24: RuntimeWarning: Mean of empty slice
  mean_value = np.nanmean(values)


,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,1.0,1.0,52.31,36.16,41.72,8.55
1,H-UNet-BASE-SWTA-T,0,1.0,1.0,45.60,30.22,44.54,9.98
2,H-UNet-HPCA,0,1.0,1.0,33.72,20.47,NaN,NaN
3,H-UNet-HPCA-T,0,1.0,1.0,34.84,21.30,NaN,NaN


/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a

Dice         Jaccard          \
                                     Mean CI 0.9%    Mean CI 0.9%   
Model              Inv Temp Regime                                  
H-UNet-BASE-SWTA   1.0      1.0     52.31     NaN   36.16     NaN   
H-UNet-BASE-SWTA-T 1.0      1.0     45.60     NaN   30.22     NaN   
H-UNet-HPCA        1.0      1.0     33.72     NaN   20.47     NaN   
H-UNet-HPCA-T      1.0      1.0     34.84     NaN   21.30     NaN   

                                   Hausdorff Distance          \
                                                 Mean CI 0.9%   
Model              Inv Temp Regime                              
H-UNet-BASE-SWTA   1.0      1.0                 41.72     NaN   
H-UNet-BASE-SWTA-T 1.0      1.0                 44.54     NaN   
H-UNet-HPCA        1.0      1.0                   NaN     NaN   
H-UNet-HPCA-T      1.0      1.0                   NaN     NaN   

                                   Average Surface Distance          
                                                       Mean CI 0.9%  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   1.0      1.0                        8.55     NaN  
H-UNet-BASE-SWTA-T 1.0      1.0                        9.98     NaN  
H-UNet-HPCA        1.0      1.0                         NaN     NaN  
H-UNet-HPCA-T      1.0      1.0                         NaN     NaN

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

In [42]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['0.01', '0.02', '0.05', '0.1', '0.2', '1.0']       # regimes to be considered
IMAGE_SIZE = 480
INV_TEMP_GlaS=20          # to be set accordingly, used by SWTA

runs = {
    'UNet': list(Path(EXP_ROOT + '/experiment=glas/').glob('unet_base')),
    #'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=glas').glob('unet')),
    #'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-hpca_ft')),
    #'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-hpca_t_ft')),
    #'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=glas').glob('hunet-hpca_ft')),
    #'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=glas').glob('hunet-hpca_t_ft')),
}

runs_swta = {
    #'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_ft').glob('inv_temp-{}'.format(INV_TEMP_GlaS))),
    #'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t_ft').glob('inv_temp-{}'.format(INV_TEMP_GlaS))),
    #'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    #'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
}

In [43]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE, ignore_outliers=True) for k, v in runs.items() for r in v], ignore_index=True)
if bool([a for a in runs_swta.values() if a != []]):
    predictions_swta = pd.concat([collect_all_temperature(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE, ignore_outliers=True) for k, v in runs_swta.items() for r in v], ignore_index=True)
    predictions = pd.concat([predictions, predictions_swta])

In [44]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics, confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,UNet,0,1.0,0.01,66.75,51.98,56.58,8.82
1,UNet,0,1.0,0.02,68.93,54.11,51.35,7.46
2,UNet,0,1.0,0.05,69.35,54.02,44.38,6.26
3,UNet,0,1.0,0.10,71.04,55.98,33.19,4.57
4,UNet,0,1.0,0.20,82.31,70.79,21.56,2.75
5,UNet,0,1.0,1.00,88.94,80.68,9.36,1.45
6,UNet,1,1.0,0.01,68.89,54.25,52.27,7.85
7,UNet,1,1.0,0.02,69.97,56.12,50.18,7.80
8,UNet,1,1.0,0.05,72.35,57.84,44.46,5.82
9,UNet,1,1.0,0.10,74.05,60.37,38.34,5.83


/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])


Dice           Jaccard           Hausdorff Distance  \
                         Mean   CI 0.9%    Mean   CI 0.9%               Mean   
Model Inv Temp Regime                                                          
UNet  1.0      0.01    66.862  1.335991  52.148  1.500145             55.383   
               0.02    67.843  1.427688  53.129  1.620535             50.237   
               0.05    69.953  2.696365  55.235  3.054250             41.687   
               0.10    74.390  1.497551  60.615  1.953559             34.839   
               0.20    79.676  1.109602  67.314  1.470966             26.129   
               1.00    88.724  0.229545  80.376  0.354158             11.456   

                                Average Surface Distance            
                        CI 0.9%                     Mean   CI 0.9%  
Model Inv Temp Regime                                               
UNet  1.0      0.01    2.215164                    8.879  0.664581  
               0.02    2.721977                    7.811  0.412108  
               0.05    4.782975                    6.202  0.773673  
               0.10    3.291246                    5.017  0.503963  
               0.20    2.694007                    3.541  0.432587  
               1.00    1.283130                    1.618  0.122690

<h1>PH2 Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [45]:
# Unsupervised learning models are evaluate over the whole dataset while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['1.0']
IMAGE_SIZE = 480

runs = {
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta_t')),
}

In [46]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv', regimes=REGIMES, image_size=IMAGE_SIZE, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [47]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,1.0,1.0,38.46,24.57,88.37,19.27
1,H-UNet-BASE-SWTA,0,5.0,1.0,48.69,33.11,128.40,17.75
2,H-UNet-BASE-SWTA,0,10.0,1.0,41.50,26.80,128.04,21.26
3,H-UNet-BASE-SWTA,0,20.0,1.0,50.86,35.08,118.39,16.46
4,H-UNet-BASE-SWTA,0,50.0,1.0,51.85,36.25,67.03,10.53
5,H-UNet-BASE-SWTA,0,100.0,1.0,49.54,34.06,128.33,18.33
6,H-UNet-BASE-SWTA-T,0,1.0,1.0,39.18,25.08,142.65,29.84
7,H-UNet-BASE-SWTA-T,0,5.0,1.0,47.29,31.72,128.85,18.26
8,H-UNet-BASE-SWTA-T,0,10.0,1.0,51.08,35.17,118.49,16.77
9,H-UNet-BASE-SWTA-T,0,20.0,1.0,45.40,30.08,128.75,20.37


/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a

Dice         Jaccard          \
                                     Mean CI 0.9%    Mean CI 0.9%   
Model              Inv Temp Regime                                  
H-UNet-BASE-SWTA   1.0      1.0     38.46     NaN   24.57     NaN   
                   5.0      1.0     48.69     NaN   33.11     NaN   
                   10.0     1.0     41.50     NaN   26.80     NaN   
                   20.0     1.0     50.86     NaN   35.08     NaN   
                   50.0     1.0     51.85     NaN   36.25     NaN   
                   100.0    1.0     49.54     NaN   34.06     NaN   
H-UNet-BASE-SWTA-T 1.0      1.0     39.18     NaN   25.08     NaN   
                   5.0      1.0     47.29     NaN   31.72     NaN   
                   10.0     1.0     51.08     NaN   35.17     NaN   
                   20.0     1.0     45.40     NaN   30.08     NaN   
                   50.0     1.0     51.11     NaN   35.35     NaN   
                   100.0    1.0     53.49     NaN   37.81     NaN   

                                   Hausdorff Distance          \
                                                 Mean CI 0.9%   
Model              Inv Temp Regime                              
H-UNet-BASE-SWTA   1.0      1.0                 88.37     NaN   
                   5.0      1.0                128.40     NaN   
                   10.0     1.0                128.04     NaN   
                   20.0     1.0                118.39     NaN   
                   50.0     1.0                 67.03     NaN   
                   100.0    1.0                128.33     NaN   
H-UNet-BASE-SWTA-T 1.0      1.0                142.65     NaN   
                   5.0      1.0                128.85     NaN   
                   10.0     1.0                118.49     NaN   
                   20.0     1.0                128.75     NaN   
                   50.0     1.0                108.32     NaN   
                   100.0    1.0                 92.96     NaN   

                                   Average Surface Distance          
                                                       Mean CI 0.9%  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   1.0      1.0                       19.27     NaN  
                   5.0      1.0                       17.75     NaN  
                   10.0     1.0                       21.26     NaN  
                   20.0     1.0                       16.46     NaN  
                   50.0     1.0                       10.53     NaN  
                   100.0    1.0                       18.33     NaN  
H-UNet-BASE-SWTA-T 1.0      1.0                       29.84     NaN  
                   5.0      1.0                       18.26     NaN  
                   10.0     1.0                       16.77     NaN  
                   20.0     1.0                       20.37     NaN  
                   50.0     1.0                       14.29     NaN  
                   100.0    1.0                       12.72     NaN

In [48]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['0.01', '0.02', '0.05', '0.1', '0.2']
IMAGE_SIZE = 480

runs = {
    'H-UNet-BASE-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta_ft')),
    'H-UNet-BASE-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta_t_ft')),
    'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet-swta_ft')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet-swta_t_ft')),
}

In [49]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [50]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA-FT,0,1.0,0.01,67.89,53.01,33.65,6.16
1,H-UNet-BASE-SWTA-FT,0,1.0,0.02,80.03,68.27,20.57,3.14
2,H-UNet-BASE-SWTA-FT,0,1.0,0.05,81.05,69.37,18.56,2.40
3,H-UNet-BASE-SWTA-FT,0,1.0,0.10,81.11,69.60,11.03,1.64
4,H-UNet-BASE-SWTA-FT,0,1.0,0.20,87.10,78.26,14.26,2.13
...,...,...,...,...,...,...,...,...
1165,H-UNet-Pseudolabeling-SWTA-T-FT,9,50.0,0.10,88.61,80.39,14.51,2.10
1166,H-UNet-Pseudolabeling-SWTA-T-FT,9,100.0,0.01,63.90,54.71,75.11,21.44
1167,H-UNet-Pseudolabeling-SWTA-T-FT,9,100.0,0.02,83.68,74.96,19.20,3.39
1168,H-UNet-Pseudolabeling-SWTA-T-FT,9,100.0,0.05,86.64,78.35,16.46,2.48


/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])


Dice           Jaccard  \
                                                   Mean   CI 0.9%    Mean   
Model                           Inv Temp Regime                             
H-UNet-BASE-SWTA-FT             1.0      0.01    69.108  3.332321  56.034   
                                         0.02    79.365  2.132228  67.474   
                                         0.05    79.324  1.531145  67.333   
                                         0.10    82.191  1.638170  70.886   
                                         0.20    84.971  1.217873  74.865   
...                                                 ...       ...     ...   
H-UNet-Pseudolabeling-SWTA-T-FT 50.0     0.10    88.333  1.383919  80.335   
                                100.0    0.01    74.314  3.960167  63.876   
                                         0.02    80.598  3.135147  71.400   
                                         0.05    85.798  1.539773  77.046   
                                         0.10    88.359  1.550463  80.298   

                                                          Hausdorff Distance  \
                                                  CI 0.9%               Mean   
Model                           Inv Temp Regime                                
H-UNet-BASE-SWTA-FT             1.0      0.01    3.837842             53.191   
                                         0.02    2.546203             20.693   
                                         0.05    2.087435             19.674   
                                         0.10    2.286052             15.291   
                                         0.20    1.543937             13.357   
...                                                   ...                ...   
H-UNet-Pseudolabeling-SWTA-T-FT 50.0     0.10    1.877844             14.662   
                                100.0    0.01    4.651427             54.657   
                                         0.02    3.559263             31.792   
                                         0.05    1.968156             19.493   
                                         0.10    2.078134             15.031   

                                                            \
                                                   CI 0.9%   
Model                           Inv Temp Regime              
H-UNet-BASE-SWTA-FT             1.0      0.01    15.067495   
                                         0.02     4.358541   
                                         0.05     3.583420   
                                         0.10     5.426397   
                                         0.20     5.084979   
...                                                    ...   
H-UNet-Pseudolabeling-SWTA-T-FT 50.0     0.10     3.586553   
                                100.0    0.01    16.876555   
                                         0.02     8.662330   
                                         0.05     4.574220   
                                         0.10     6.197934   

                                                Average Surface Distance  \
                                                                    Mean   
Model                           Inv Temp Regime                            
H-UNet-BASE-SWTA-FT             1.0      0.01                      9.968   
                                         0.02                      3.227   
                                         0.05                      2.907   
                                         0.10                      1.957   
                                         0.20                      1.982   
...                                                                  ...   
H-UNet-Pseudolabeling-SWTA-T-FT 50.0     0.10                      2.069   
                                100.0    0.01                     10.160   
                                         0.02                      5.411   
                                         0.05                      2.8

<h2>Evaluation - Hebbian Unsupervised Pretraining</h2>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [51]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

IMAGE_SIZE = 480
INV_TEMP_PH2=100          # to be set accordingly, used by SWTA

runs = {
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-hpca/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-hpca_t/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run-*')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run-*')),
}

In [52]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv', image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [53]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

/tmp/ipykernel_1451037/3550580355.py:24: RuntimeWarning: Mean of empty slice
  mean_value = np.nanmean(values)


,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,100.0,1.0,49.54,34.06,128.33,18.33
1,H-UNet-BASE-SWTA-T,0,100.0,1.0,53.49,37.81,92.96,12.72
2,H-UNet-HPCA,0,1.0,1.0,21.33,12.19,246.21,86.46
3,H-UNet-HPCA-T,0,1.0,1.0,21.98,12.62,NaN,NaN


/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a

Dice         Jaccard          \
                                     Mean CI 0.9%    Mean CI 0.9%   
Model              Inv Temp Regime                                  
H-UNet-BASE-SWTA   100.0    1.0     49.54     NaN   34.06     NaN   
H-UNet-BASE-SWTA-T 100.0    1.0     53.49     NaN   37.81     NaN   
H-UNet-HPCA        1.0      1.0     21.33     NaN   12.19     NaN   
H-UNet-HPCA-T      1.0      1.0     21.98     NaN   12.62     NaN   

                                   Hausdorff Distance          \
                                                 Mean CI 0.9%   
Model              Inv Temp Regime                              
H-UNet-BASE-SWTA   100.0    1.0                128.33     NaN   
H-UNet-BASE-SWTA-T 100.0    1.0                 92.96     NaN   
H-UNet-HPCA        1.0      1.0                246.21     NaN   
H-UNet-HPCA-T      1.0      1.0                   NaN     NaN   

                                   Average Surface Distance          
                                                       Mean CI 0.9%  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   100.0    1.0                       18.33     NaN  
H-UNet-BASE-SWTA-T 100.0    1.0                       12.72     NaN  
H-UNet-HPCA        1.0      1.0                       86.46     NaN  
H-UNet-HPCA-T      1.0      1.0                         NaN     NaN

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

In [54]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['0.01', '0.02', '0.05', '0.1', '0.2', '1.0']       # regimes to be considered
IMAGE_SIZE = 480
INV_TEMP_PH2=100          # to be set accordingly, used by SWTA

runs = {
    'UNet': list(Path(EXP_ROOT + '/experiment=ph2/').glob('unet_base')),
    #'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=ph2').glob('unet')),
    #'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-hpca_ft')),
    #'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-hpca_t_ft')),
    #'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=ph2').glob('hunet-hpca_ft')),
    #'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2').glob('hunet-hpca_t_ft')),
}

runs_swta = {
    #'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta_ft').glob('inv_temp-{}'.format(INV_TEMP_PH2))),
    #'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta_t_ft').glob('inv_temp-{}'.format(INV_TEMP_PH2))),
    #'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    #'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
}

In [55]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)
if bool([a for a in runs_swta.values() if a != []]):
    predictions_swta = pd.concat([collect_all_temperature(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE) for k, v in runs_swta.items() for r in v], ignore_index=True)
    predictions = pd.concat([predictions, predictions_swta])

In [56]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,UNet,0,1.0,0.01,82.10,71.65,19.81,3.04
1,UNet,0,1.0,0.02,77.74,65.58,23.60,4.04
2,UNet,0,1.0,0.05,74.41,62.28,45.76,7.49
3,UNet,0,1.0,0.10,85.51,75.59,7.23,0.99
4,UNet,0,1.0,0.20,79.33,67.10,14.08,1.87
5,UNet,0,1.0,1.00,89.11,81.31,12.49,1.77
6,UNet,1,1.0,0.01,71.05,60.77,53.91,10.17
7,UNet,1,1.0,0.02,67.47,55.72,67.38,12.53
8,UNet,1,1.0,0.05,73.60,60.44,52.82,7.93
9,UNet,1,1.0,0.10,74.55,61.64,47.74,7.36


/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])


Dice           Jaccard           Hausdorff Distance  \
                         Mean   CI 0.9%    Mean   CI 0.9%               Mean   
Model Inv Temp Regime                                                          
UNet  1.0      0.01    73.764  2.927977  62.455  3.437301             49.300   
               0.02    74.482  3.562861  62.332  4.249428             36.842   
               0.05    77.064  2.173455  65.028  2.813769             30.054   
               0.10    80.056  2.133722  68.528  2.567291             19.371   
               0.20    84.177  1.276208  73.532  1.777131             11.118   
               1.00    90.982  0.940720  83.943  1.431540              6.673   

                                 Average Surface Distance            
                         CI 0.9%                     Mean   CI 0.9%  
Model Inv Temp Regime                                                
UNet  1.0      0.01    13.638165                    8.763  2.684645  
               0.02    10.258153                    5.926  1.711405  
               0.05     7.139774                    4.676  1.204499  
               0.10     6.962100                    3.029  1.175288  
               0.20     3.234734                    1.440  0.309052  
               1.00     3.755556                    1.016  0.522339

<h1>TREND Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [57]:
# Unsupervised learning models are evaluate over the whole dataset while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['1.0']
IMAGE_SIZE = 480

runs = {
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=trend/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=trend/').glob('hunet_base-swta_t')),
}

In [58]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv', regimes=REGIMES, image_size=IMAGE_SIZE, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [59]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

/tmp/ipykernel_1451037/3550580355.py:24: RuntimeWarning: Mean of empty slice
  mean_value = np.nanmean(values)


,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,1.0,1.0,6.23,3.22,NaN,NaN
1,H-UNet-BASE-SWTA,0,5.0,1.0,6.37,3.29,NaN,NaN
2,H-UNet-BASE-SWTA,0,10.0,1.0,6.40,3.31,NaN,NaN
3,H-UNet-BASE-SWTA,0,20.0,1.0,6.46,3.34,NaN,NaN
4,H-UNet-BASE-SWTA,0,50.0,1.0,6.53,3.38,NaN,NaN
5,H-UNet-BASE-SWTA,0,100.0,1.0,6.82,3.54,354.26,113.95
6,H-UNet-BASE-SWTA-T,0,1.0,1.0,6.18,3.19,NaN,NaN
7,H-UNet-BASE-SWTA-T,0,5.0,1.0,6.77,3.51,NaN,NaN
8,H-UNet-BASE-SWTA-T,0,10.0,1.0,6.99,3.62,286.18,84.99
9,H-UNet-BASE-SWTA-T,0,20.0,1.0,7.26,3.77,311.13,105.10


/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a

Dice         Jaccard          \
                                    Mean CI 0.9%    Mean CI 0.9%   
Model              Inv Temp Regime                                 
H-UNet-BASE-SWTA   1.0      1.0     6.23     NaN    3.22     NaN   
                   5.0      1.0     6.37     NaN    3.29     NaN   
                   10.0     1.0     6.40     NaN    3.31     NaN   
                   20.0     1.0     6.46     NaN    3.34     NaN   
                   50.0     1.0     6.53     NaN    3.38     NaN   
                   100.0    1.0     6.82     NaN    3.54     NaN   
H-UNet-BASE-SWTA-T 1.0      1.0     6.18     NaN    3.19     NaN   
                   5.0      1.0     6.77     NaN    3.51     NaN   
                   10.0     1.0     6.99     NaN    3.62     NaN   
                   20.0     1.0     7.26     NaN    3.77     NaN   
                   50.0     1.0     7.71     NaN    4.01     NaN   
                   100.0    1.0     7.42     NaN    3.86     NaN   

                                   Hausdorff Distance          \
                                                 Mean CI 0.9%   
Model              Inv Temp Regime                              
H-UNet-BASE-SWTA   1.0      1.0                   NaN     NaN   
                   5.0      1.0                   NaN     NaN   
                   10.0     1.0                   NaN     NaN   
                   20.0     1.0                   NaN     NaN   
                   50.0     1.0                   NaN     NaN   
                   100.0    1.0                354.26     NaN   
H-UNet-BASE-SWTA-T 1.0      1.0                   NaN     NaN   
                   5.0      1.0                   NaN     NaN   
                   10.0     1.0                286.18     NaN   
                   20.0     1.0                311.13     NaN   
                   50.0     1.0                287.52     NaN   
                   100.0    1.0                312.26     NaN   

                                   Average Surface Distance          
                                                       Mean CI 0.9%  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   1.0      1.0                         NaN     NaN  
                   5.0      1.0                         NaN     NaN  
                   10.0     1.0                         NaN     NaN  
                   20.0     1.0                         NaN     NaN  
                   50.0     1.0                         NaN     NaN  
                   100.0    1.0                      113.95     NaN  
H-UNet-BASE-SWTA-T 1.0      1.0                         NaN     NaN  
                   5.0      1.0                         NaN     NaN  
                   10.0     1.0                       84.99     NaN  
                   20.0     1.0                      105.10     NaN  
                   50.0     1.0                       91.05     NaN  
                   100.0    1.0                       79.65     NaN

In [63]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['0.02', '0.05', '0.1', '0.2']
IMAGE_SIZE = 480

runs = {
    'H-UNet-BASE-SWTA-FT': list(Path(EXP_ROOT + '/experiment=trend/').glob('hunet_base-swta_ft')),
    'H-UNet-BASE-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=trend/').glob('hunet_base-swta_t_ft')),
    'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=trend/').glob('hunet-swta_ft')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=trend/').glob('hunet-swta_t_ft')),
}

In [64]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [65]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA-FT,0,1.0,0.05,24.63,14.10,81.74,13.40
1,H-UNet-BASE-SWTA-FT,0,1.0,0.10,34.27,20.72,21.13,3.17
2,H-UNet-BASE-SWTA-FT,0,1.0,0.20,39.78,24.88,15.90,2.35
3,H-UNet-BASE-SWTA-FT,0,10.0,0.02,32.42,19.39,19.35,2.87
4,H-UNet-BASE-SWTA-FT,0,10.0,0.05,34.62,20.99,16.73,2.53
...,...,...,...,...,...,...,...,...
234,H-UNet-BASE-SWTA-T-FT,9,10.0,0.20,43.87,28.14,20.16,2.66
235,H-UNet-BASE-SWTA-T-FT,9,50.0,0.02,28.10,16.38,31.67,4.69
236,H-UNet-BASE-SWTA-T-FT,9,50.0,0.05,34.57,20.94,19.98,2.87
237,H-UNet-BASE-SWTA-T-FT,9,50.0,0.10,39.63,24.77,19.56,2.71


/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])


Dice              Jaccard  \
                                            Mean   CI 0.9%       Mean   
Model                 Inv Temp Regime                                   
H-UNet-BASE-SWTA-FT   1.0      0.02    22.235556  1.427593  12.565556   
                               0.05    21.060000  1.638229  11.835000   
                               0.10    32.655000  1.300713  19.586000   
                               0.20    37.639000  0.717080  23.254000   
                      10.0     0.02    29.252000  1.311002  17.186000   
                               0.05    31.419000  0.916741  18.689000   
                               0.10    38.846000  0.923733  24.161000   
                               0.20    43.750000  0.721590  28.062000   
                      50.0     0.02    31.661000  0.712383  18.865000   
                               0.05    33.352000  0.921083  20.067000   
                               0.10    40.328000  0.823967  25.316000   
                               0.20    45.272000  0.404448  29.317000   
H-UNet-BASE-SWTA-T-FT 1.0      0.02    20.246000  2.098313  11.343000   
                               0.05    23.260000  1.558079  13.237000   
                               0.10    33.076000  0.846927  19.881000   
                               0.20    38.697000  0.777841  24.064000   
                      10.0     0.02    30.560000  1.225022  18.098000   
                               0.05    31.735000  0.787877  18.908000   
                               0.10    38.950000  0.544160  24.240000   
                               0.20    43.483000  0.763337  27.842000   
                      50.0     0.02    31.195000  1.249947  18.540000   
                               0.05    32.318000  0.794852  19.320000   
                               0.10    38.525000  1.214573  23.921000   
                               0.20    43.970000  0.663673  28.242000   

                                                Hausdorff Distance             \
                                        CI 0.9%               Mean    CI 0.9%   
Model                 Inv Temp Regime                                           
H-UNet-BASE-SWTA-FT   1.0      0.02    0.910657         108.903333  19.957747   
                               0.05    1.027826         122.058000  22.305929   
                               0.10    0.917924          37.536000  10.272998   
                               0.20    0.544556          23.330000   2.317725   
                      10.0     0.02    0.895048          29.279000   7.717465   
                               0.05    0.651194          27.333000   4.743971   
                               0.10    0.711037          19.146000   2.727046   
                               0.20    0.593039          17.754000   2.889537   
                      50.0     0.02    0.504691          24.242000   3.421773   
                               0.05    0.676739          20.785000   3.123256   
                               0.10    0.652558          17.952000   1.593189   
                               0.20    0.333849          16.003000   1.395511   
H-UNet-BASE-SWTA-T-FT 1.0      0.02    1.307946         137.164000  26.778353   
                               0.05    1.001477         103.827000  20.502788   
                               0.10    0.610013          41.132000   4.983784   
                               0.20    0.597618          25.060000   1.332990   
                      10.0     0.02    0.857728          29.221000   5.565442   
                               0.05    0.557235          26.500000   3.498970   
                               0.10    0.421050          19.598000   1.987988   
                               0.20    0.616274          17.858000   2.087380   
                      50.0     0.02    0.884898          29.477000   6.184229   
                               0.05    0.564021          23.586000   3.077066   
                               0.10    0.946016

<h2>Evaluation - Hebbian Unsupervised Pretraining</h2>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [66]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

IMAGE_SIZE = 480
INV_TEMP_TREND=50          # to be set accordingly, used by SWTA

runs = {
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=trend/hunet_base-hpca/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=trend/hunet_base-hpca_t/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=trend/hunet_base-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_TREND)).glob('run-*')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=trend/hunet_base-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_TREND)).glob('run-*')),
}

In [67]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv', image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [68]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

/tmp/ipykernel_1451037/3550580355.py:24: RuntimeWarning: Mean of empty slice
  mean_value = np.nanmean(values)


,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,50.0,1.0,6.53,3.38,NaN,NaN
1,H-UNet-BASE-SWTA-T,0,50.0,1.0,7.71,4.01,287.52,91.05
2,H-UNet-HPCA,0,1.0,1.0,3.34,1.70,NaN,NaN
3,H-UNet-HPCA-T,0,1.0,1.0,3.25,1.65,NaN,NaN


/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/hebbian-segmentation/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a

Dice         Jaccard          \
                                    Mean CI 0.9%    Mean CI 0.9%   
Model              Inv Temp Regime                                 
H-UNet-BASE-SWTA   50.0     1.0     6.53     NaN    3.38     NaN   
H-UNet-BASE-SWTA-T 50.0     1.0     7.71     NaN    4.01     NaN   
H-UNet-HPCA        1.0      1.0     3.34     NaN    1.70     NaN   
H-UNet-HPCA-T      1.0      1.0     3.25     NaN    1.65     NaN   

                                   Hausdorff Distance          \
                                                 Mean CI 0.9%   
Model              Inv Temp Regime                              
H-UNet-BASE-SWTA   50.0     1.0                   NaN     NaN   
H-UNet-BASE-SWTA-T 50.0     1.0                287.52     NaN   
H-UNet-HPCA        1.0      1.0                   NaN     NaN   
H-UNet-HPCA-T      1.0      1.0                   NaN     NaN   

                                   Average Surface Distance          
                                                       Mean CI 0.9%  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   50.0     1.0                         NaN     NaN  
H-UNet-BASE-SWTA-T 50.0     1.0                       91.05     NaN  
H-UNet-HPCA        1.0      1.0                         NaN     NaN  
H-UNet-HPCA-T      1.0      1.0                         NaN     NaN

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

In [70]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['0.02', '0.05', '0.1', '0.2', '1.0']       # regimes to be considered
IMAGE_SIZE = 480
INV_TEMP_TREND=1          # to be set accordingly, used by SWTA

runs = {
    'UNet': list(Path(EXP_ROOT + '/experiment=trend/').glob('unet_base')),
    #'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=trend').glob('unet')),
    #'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=trend/').glob('hunet_base-hpca_ft')),
    #'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=trend/').glob('hunet_base-hpca_t_ft')),
    #'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=trend').glob('hunet-hpca_ft')),
    #'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=trend').glob('hunet-hpca_t_ft')),
}

runs_swta = {
    #'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=trend/hunet_base-swta_ft').glob('inv_temp-{}'.format(INV_TEMP_TREND))),
    #'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=trend/hunet_base-swta_t_ft').glob('inv_temp-{}'.format(INV_TEMP_TREND))),
    #'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=trend/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_TREND)).glob('regime-*')),
    #'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=trend/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_TREND)).glob('regime-*')),
}

In [71]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE, ignore_outliers=True) for k, v in runs.items() for r in v], ignore_index=True)
if bool([a for a in runs_swta.values() if a != []]):
    predictions_swta = pd.concat([collect_all_temperature(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE, ignore_outliers=True) for k, v in runs_swta.items() for r in v], ignore_index=True)
    predictions = pd.concat([predictions, predictions_swta])

In [72]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,UNet,0,1.0,0.02,38.59,23.96,16.74,2.42
1,UNet,0,1.0,0.05,38.23,23.71,28.43,4.04
2,UNet,0,1.0,0.10,40.95,25.82,13.66,2.13
3,UNet,0,1.0,0.20,42.90,27.36,14.12,2.17
4,UNet,0,1.0,1.00,50.15,33.50,13.78,1.83
5,UNet,1,1.0,0.02,34.92,21.20,45.17,6.23
6,UNet,1,1.0,0.05,32.14,19.19,46.90,6.76
7,UNet,1,1.0,0.10,38.85,24.18,22.36,3.15
8,UNet,1,1.0,0.20,43.34,27.78,11.95,1.97
9,UNet,1,1.0,1.00,49.58,33.03,14.92,2.03


/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])
/tmp/ipykernel_1451037/3550580355.py:50: FutureWarning: The provided callable <function mean at 0x7f7a50119e10> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ("CI {}%".format(confidence_level), compute_ci)])


Dice              Jaccard            \
                            Mean   CI 0.9%       Mean   CI 0.9%   
Model Inv Temp Regime                                             
UNet  1.0      0.02    35.353333  1.407369  21.547778  1.038271   
               0.05    35.519000  1.392105  21.681000  1.030263   
               0.10    38.558000  1.208072  23.968000  0.934096   
               0.20    42.659000  0.706153  27.179000  0.577785   
               1.00    50.393000  0.632294  33.748000  0.572224   

                      Hausdorff Distance           Average Surface Distance  \
                                    Mean   CI 0.9%                     Mean   
Model Inv Temp Regime                                                         
UNet  1.0      0.02            42.168889  9.782490                 6.084444   
               0.05            38.121000  4.276814                 5.326000   
               0.10            26.957000  6.056765                 3.769000   
               0.20            18.148000  2.566620                 2.594000   
               1.00            13.194000  1.413341                 1.785000   

                                 
                        CI 0.9%  
Model Inv Temp Regime            
UNet  1.0      0.02    1.504435  
               0.05    0.593534  
               0.10    0.788407  
               0.20    0.300040  
               1.00    0.142164